In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_squared_error
import kerastuner as kt
import pickle
import joblib
from scipy import stats
from numpy import load
from numpy import save
from tensorflow.keras import regularizers
import seaborn as sns

In [ ]:
sns.set_style("white")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

### Set seed

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

### Load Data

In [ ]:
train_data = pd.read_pickle("../final_data/train_unsc_mlp.pkl")
valid_data = pd.read_pickle("../final_data/valid_unsc_mlp.pkl")
test_data = pd.read_pickle("../final_data/test_unsc_mlp.pkl")
all_train_data = pd.read_pickle("/../final_data/all_train_unsc_mlp.pkl")

train_x = load('../final_data/train_x_mlp.npy')
train_y = load('../final_data/train_y_mlp.npy')
valid_x = load('../final_data/valid_x_mlp.npy')
valid_y = load('../final_data/valid_y_mlp.npy')
test_x = load('../final_data/test_x_mlp.npy')
test_y = load('../final_data/test_y_mlp.npy')
all_train_x = load('../final_data/all_x_mlp.npy')
all_train_y = load('../final_data/all_y_mlp.npy')

with open ('../final_data/label_keys_mlp.pkl', 'rb') as fp:
    label_keys = pickle.load(fp)
with open ('../final_data/feature_keys_mlp.pkl', 'rb') as fp:
    feature_keys = pickle.load(fp)    

### Load Scaler

In [ ]:
scaler_y = joblib.load("../final_data/scaler_y_mlp.save") 
scaler_y_train = joblib.load("../final_data/scaler_y_train_mlp.save") 

## Loop function

In [ ]:
def test_10_times(model, epochs, batch_size):
    runs = 10
    test_score = list()
    train_score = list()
    for i in range(runs):

        np.random.seed(42 + i)
        tf.random.set_seed(42 + i)

        history = model.fit(all_train_x, all_train_y,
                        epochs=epochs,
                        batch_size=batch_size,
                       )

        y_pred_test = model.predict(test_x)
        y_test_unsc = scaler_y.inverse_transform(test_y)
        y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)

        y_pred_train = model.predict(all_train_x)
        y_train_unsc = scaler_y.inverse_transform(all_train_y)
        y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
        
        mse_test = np.sqrt(mean_squared_error(y_pred_test_unsc, y_test_unsc))  
        test_score.append(mse_test)

        mse_train = np.sqrt(mean_squared_error(y_pred_train_unsc, y_train_unsc))  
        train_score.append(mse_train)

    return  train_score, test_score


### Different Models

#### Simple MLP

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

def build_model_simple_mlp():
    model = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:]),
        keras.layers.Dense(2)
    ])
    model.compile(loss='mse',           
                  optimizer=keras.optimizers.Adam(lr=1e-3),           
                 )
    return model

### Training

In [ ]:
simple_mlp = build_model_simple_mlp()

simple_mlp_history = simple_mlp.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                   )

In [ ]:
simple_mlp = keras.models.load_model("../final_data/models/simple_mlp_train")
hist_simple_mlp = pd.read_pickle("../final_data/hist_simple_mlp_train.pkl")

#### Performance

In [ ]:
pd.DataFrame(hist_simple_mlp).plot(figsize=(8, 5))
plt.grid(True)

In [ ]:
## Calculate RMSE on training set
y_pred_train = simple_mlp.predict(train_x)
y_train_unsc = scaler_y_train.inverse_transform(train_y)
y_pred_train_unsc = scaler_y_train.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
# calculate RMSE on validation set
y_pred_val = simple_mlp.predict(valid_x)
y_val_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_val_unsc = scaler_y_train.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing

In [ ]:
simple_mlp = build_model_simple_mlp()

simple_mlp_history = simple_mlp.fit(all_train_x, all_train_y,
                    epochs=160,
                    batch_size=32,
                   )

In [ ]:
simple_mlp = keras.models.load_model("../final_data/models/simple_mlp")

In [ ]:
## Calculate RMSE on all training data
y_pred_all_train = simple_mlp.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
## Caculate RMSE on test set
y_pred_test = simple_mlp.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))         

In [ ]:
simple_mlp = build_model_simple_mlp()
epochs = 160
batch_size = 32
train_score, test_score = test_10_times(simple_mlp, epochs, batch_size)

### Test 10 times

In [ ]:
rmse_train_simple_mlp = pd.read_pickle("../final_data/rmse_train_simple_mlp.pkl")
rmse_test_simple_mlp = pd.read_pickle("../final_data/rmse_test_simple_mlp.pkl")

In [ ]:
rmse_train_simple_mlp.describe()

In [ ]:
rmse_test_simple_mlp.describe()

## Simple Model with Dropout

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=20)

def build_model_simple_dropout():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:]),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
                 
        optimizer=keras.optimizers.Adam(lr=1e-3),
                  
                 )
    return model_reg

### Training

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

model_simple_dropout = build_model_simple_dropout()

history_simple_dropout = model_simple_dropout.fit(train_x, train_y,
                    epochs=200,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )


Load Model

In [ ]:
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_mlp_drop.pkl")
model_simple_dropout = keras.models.load_model("../final_data/models/simple_mlp_drop")

In [ ]:
pd.DataFrame(hist_simple_dropout).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.004)
plt.show()

In [ ]:
y_pred_train = model_simple_dropout.predict(train_x)
y_train_unsc = scaler_y_train.inverse_transform(train_y)
y_pred_train_unsc = scaler_y_train.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_simple_dropout.predict(valid_x)
y_val_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_val_unsc = scaler_y_train.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

## Simple Model with Weight Regularization

In [ ]:
def build_model_simple_weight_reg():
    model_kernel = keras.models.Sequential([
        keras.layers.Dense(32, activation='relu',kernel_initializer='he_normal', input_shape=train_x.shape[1:],
                          kernel_regularizer=regularizers.l2(0.001)),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_kernel.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(lr=1e-3),
                 
                 )
    return model_kernel

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

model_simple_weight_reg = build_model_simple_weight_reg()

history_simple_weight_reg = model_simple_weight_reg.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load Model:

In [ ]:
hist_simple_weight_reg = pd.read_pickle("../final_data/hist_simple_mlp_weight.pkl")
model_simple_weight_reg = keras.models.load_model("../final_data/models/simple_mlp_weight_reg")

In [ ]:
pd.DataFrame(hist_simple_weight_reg).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 0.004)
plt.show()

In [ ]:
y_pred_train = model_simple_weight_reg.predict(train_x)
y_train_unsc = scaler_y_train.inverse_transform(train_y)
y_pred_train_unsc = scaler_y_train.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = model_simple_weight_reg.predict(valid_x)
y_val_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_val_unsc = scaler_y_train.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### MLP Dropout Hyperband

### Keras Tuner implementation of Hyperband
Pickle containing all the models Hyperband testes can be loaded and therefore the best model found as well. So there is no need to run the Hyperparameter Search with Hyperband

In order to load the pickle with the Hyperband search results, the following cell must be run!

In [ ]:
def model_builder_mlp (hp):
    
    model = keras.models.Sequential()
    hp_units = hp.Int("unit",min_value=32,max_value=160,step=32)
    hp_dropout = hp.Choice("mlp_layer_dropout",values = [ 0.2, 0.3])
    hp_number_of_layers = hp.Int("layers",min_value=0,max_value=2,step=1)
    
    model.add(keras.layers.Dense(
                            units= hp_units, 
                            kernel_constraint=keras.constraints.max_norm(max_value=1),                    
                            activation='relu',
                            
                            input_shape=train_x.shape[1:]     
                     ))
    
    model.add(keras.layers.Dropout(hp_dropout))
    
  

    for i in range(hp_number_of_layers):
        model.add(keras.layers.Dense(
                        units = hp_units,
                        kernel_constraint=keras.constraints.max_norm(max_value=1),
                        activation='relu',
                        kernel_initializer='he_normal'
                        ))
        model.add(keras.layers.Dropout(hp_dropout))
        
        

    model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
    mlp_learning_rate = hp.Choice('learning_rate', values = [ 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(lr=mlp_learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

In [ ]:
class MyTuner(kt.tuners.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

This must only be executed if Hyperband should search again!

In [ ]:
# tuner = MyTuner(model_builder_mlp,
#                 objective='val_loss',
#                 max_epochs = 100,
#                 factor = 3,
#                 seed = 42,
#                 directory = "k",
#                 project_name = 'k3')

# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

# tuner.search(train_x,
#              train_y,
#              epochs = 150,
#              validation_data = (valid_x, valid_y),
#              callbacks = [stop_early]
#             )

Load the pickle of the Hyperband search

In [ ]:
tuner_hyper_dropout = pickle.load(open("../final_data/models/mlp_drop_hyperband.pkl",'rb'))

print(tuner_hyper_dropout.get_best_hyperparameters()[0].values)

### Training

#### Either: Keras Tuner implementation of Hyperband

In [ ]:
best_hps = tuner_hyper_dropout.get_best_hyperparameters(num_trials=1)[0]
mlp_dropout_train = tuner_hyper_dropout.hypermodel.build(best_hps)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Retrain the model
history_mlp_dropout_train = mlp_dropout_train.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[early_stop]
                   )

#### Or: Rebuild best Hyperband model manually

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

def build_model_dropout_tuned():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                           input_shape=train_x.shape[1:],
                           kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                          kernel_constraint=keras.constraints.max_norm(max_value=1),),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
        optimizer=keras.optimizers.Adam(lr=1e-3),     
                 )
    return model_reg

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

mlp_dropout_train = build_model_dropout_tuned()

history_mlp_dropout_train = mlp_dropout.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load trained Hyperband model

In [ ]:
hist_dropout_train = pd.read_pickle("../final_data/hist_mlp_dropout_hyper_train.pkl")
mlp_dropout_train = keras.models.load_model("../final_data/models/mlp_dropout_hyper_train")

In [ ]:
pd.DataFrame(hist_dropout).plot(figsize=(8, 5))
plt.grid(True)
plt.show()

In [ ]:
y_pred_train = mlp_dropout.predict(train_x)
y_train_unsc = scaler_y_train.inverse_transform(train_y)
y_pred_train_unsc = scaler_y_train.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = mlp_dropout.predict(valid_x)
y_val_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_val_unsc = scaler_y_train.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing
#### Either: rebuild Keras Tuner model

In [ ]:
best_hps = tuner_hyper_dropout.get_best_hyperparameters(num_trials=1)[0]
mlp_dropout_test = tuner_hyper_dropout.hypermodel.build(best_hps)

# Retrain the model
history_mlp_dropout_test = mlp_dropout_test.fit(all_train_x, all_train_y,
                    epochs=171,
                    batch_size=32,
                   )

#### Or: rebuild best model manually

In [ ]:
mlp_dropout_test = build_model_dropout_tuned()

history_mlp_dropout_test = mlp_dropout_test.fit(all_train_x, all_train_y,
                    epochs=171,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load Model

In [ ]:
mlp_dropout_test = keras.models.load_model("../final_data/models/mlp_dropout_test")

In [ ]:
y_pred_all_train = mlp_dropout_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
y_pred_test = mlp_dropout_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))         

### Test 10 times

In [ ]:
LSTMSimple_model = build_model_dropout_tuned()
epochs = 171
batch_size = 32
train_score, test_score = test_10_times(LSTMSimple_model, epochs, batch_size)

In [ ]:
rmse_train_mlp_dropout_tuned = pd.read_pickle("../final_data/rmse_train_mlp_dropout_tuned.pkl")
rmse_test_mlp_dropout_tuned = pd.read_pickle("../final_data/rmse_test_mlp_dropout_tuned.pkl")

In [ ]:
rmse_train_mlp_dropout_tuned.describe()

In [ ]:
rmse_test_mlp_dropout_tuned.describe()

### MLP Weight Regularization Hyperband

### Keras Tuner implementation of Hyperband

### Training

In [ ]:
def model_builder_mlp (hp):
    
    model = keras.models.Sequential()
    hp_kernel_reg = hp.Choice("mlp_layer_kernel_reg",values = [1e-4, 1e-3])
    hp_units = hp.Int("unit",min_value=32,max_value=160,step=32)
    hp_number_of_layers = hp.Int("layers",min_value=0,max_value=2,step=1)
    
    model.add(keras.layers.Dense(
                            units= hp_units, 
                            kernel_regularizer=regularizers.l2(hp_kernel_reg),                  
                            activation='relu',
                            input_shape=train_x.shape[1:]     
                     ))
    
    for i in range(hp_number_of_layers):
        model.add(keras.layers.Dense(
                        units = hp_units,
                        kernel_regularizer=regularizers.l2(hp_kernel_reg),
                        activation='relu',
                        kernel_initializer='he_normal'
                        ))
        

    model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
    mlp_learning_rate = hp.Choice('learning_rate', values = [ 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(lr=mlp_learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model

class MyTuner(kt.tuners.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
    super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
def model_builder_mlp (hp):
    
    model = keras.models.Sequential()
    hp_kernel_reg = hp.Choice("mlp_layer_kernel_reg",values = [1e-3, 1e-4])
    hp_units = hp.Choice("unit",values = [16, 32, 64, 128, 192 ]) 
    hp_number_of_layers = hp.Choice("layers",values = [0,1,2,3]) 
    
    model.add(keras.layers.Dense(
                            units= hp_units, 
                            kernel_regularizer=regularizers.l2(hp_kernel_reg),          
                            activation='relu',
                            input_shape=train_x.shape[1:]     
                     ))

    for i in range(hp_number_of_layers):
        model.add(keras.layers.Dense(
                        units = hp_units,
                        kernel_regularizer=regularizers.l2(hp_kernel_reg),
                        activation='relu',
                        kernel_initializer='he_normal'
                        ))

    model.add(keras.layers.Dense(2, kernel_initializer='he_normal'))
        
    mlp_learning_rate = hp.Choice('learning_rate', values = [ 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(lr=mlp_learning_rate)
    
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    return model
              
              
class MyTuner(kt.tuners.Hyperband):
  def run_trial(self, trial, *args, **kwargs):
    kwargs['batch_size'] = trial.hyperparameters.Choice("batch_size",values = [32, 128, 512])
    super(MyTuner, self).run_trial(trial, *args, **kwargs)
    


In [ ]:
tuner = MyTuner(model_builder_mlp,
                objective='val_loss',
                max_epochs = 100,
                factor = 3,
                seed = 42,
                directory = "/home/di40438/bachelorarbeit/data/keras_tuner",
                project_name = 'mlp_weight_reg_1')

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

tuner.search(train_x,
             train_y,
             epochs = 150,
             validation_data = (valid_x, valid_y),
             callbacks = [stop_early]
            )

#### Keras Tuner implementation of Hyperband
Hyperparameter search may take a while even with limited data, resulting architectur when used with all data is provided next

In [ ]:
tuner_hyper_weight = pickle.load(open("../final_data/models/tuner_hyper_weight.pkl",'rb'))

print(tuner_hyper_weight.get_best_hyperparameters()[0].values)

In [ ]:
best_hps = tuner_hyper_weight.get_best_hyperparameters(num_trials=1)[0]
hypermodel_weight_reg = tuner_hyper_weight.hypermodel.build(best_hps)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)

# Retrain the model
history_hypermodel_weight_reg = hypermodel_weight_reg.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[early_stop]
                   )

#### Or: Rebuild best Hyperband model manually

In [ ]:
def build_model_weight_reg_tuned():
    model_reg = keras.models.Sequential([
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                           input_shape=train_x.shape[1:],
                            kernel_regularizer=regularizers.l2(0.0001)),
        keras.layers.Dense(128, activation='relu',kernel_initializer='he_normal',
                           kernel_regularizer=regularizers.l2(0.0001)),
        keras.layers.Dense(2, kernel_initializer='he_normal')
    ])
    model_reg.compile(loss='mse',
        optimizer=keras.optimizers.Adam(lr=1e-3),     
                 )
    return model_reg

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)

mlp_weight_reg_train = build_model_weight_reg_tuned()

history_mlp_weight_reg_train = mlp_weight_reg_train.fit(train_x, train_y,
                    epochs=300,
                    batch_size=32,
                    validation_data=(valid_x, valid_y),
                    callbacks=[callback]
                   )

Load Model

In [ ]:
hist_weight_reg_train = pd.read_pickle("../final_data/hist_hypermodel_mlp_weight_reg.pkl")
mlp_weight_reg_train = keras.models.load_model("../final_data/models/hypermodel_mlp_weight_reg_train")

In [ ]:
pd.DataFrame(hist_weight_reg_train).plot(figsize=(8, 5))
plt.grid(True)

In [ ]:
y_pred_train = mlp_weight_reg_train.predict(train_x)
y_train_unsc = scaler_y_train.inverse_transform(train_y)
y_pred_train_unsc = scaler_y_train.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_val = mlp_weight_reg_train.predict(valid_x)
y_val_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_val_unsc = scaler_y_train.inverse_transform(y_pred_val)
np.sqrt(mean_squared_error(y_val_unsc, y_pred_val_unsc))                       

### Testing
#### Either: rebuild Keras Tuner model

In [ ]:
best_hps = tuner_hyper_weight.get_best_hyperparameters(num_trials=1)[0]
mlp_weight_reg_test = tuner_hyper_weight.hypermodel.build(best_hps)

# Retrain the model
history_mlp_weight_reg_test = mlp_weight_reg_test.fit(all_train_x, all_train_y,
                    epochs=167,
                    batch_size=32,
                   )

#### Or: rebuild best model manually

In [ ]:
mlp_weight_reg_test = build_model_weight_reg_tuned()

history_mlp_weight_reg_test = mlp_weight_reg_test.fit(all_train_x, all_train_y,
                    epochs=167,
                    batch_size=32,
                    )

Load model

In [ ]:
mlp_weight_reg_test = keras.models.load_model("../final_data/models/mlp_weight_reg_test")

In [ ]:
y_pred_all_train = mlp_weight_reg_test.predict(all_train_x)
y_all_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_all_train_unsc = scaler_y.inverse_transform(y_pred_all_train)
np.sqrt(mean_squared_error(y_all_train_unsc, y_pred_all_train_unsc))         

In [ ]:
y_pred_test = mlp_weight_reg_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc = scaler_y.inverse_transform(y_pred_test)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc))         

### Test 10 times

In [ ]:
mlp_weight_reg = build_model_weight_reg_tuned()
epochs = 167
batch_size = 32
train_score, test_score = test_10_times(mlp_weight_reg, epochs, batch_size)

In [ ]:
rmse_train_mlp_weight_reg = pd.read_pickle("../final_data/rmse_train_mlp_weight_reg.pkl")
rmse_test_mlp_weight_reg = pd.read_pickle("../final_data/rmse_test_mlp_weight_reg.pkl")

In [ ]:
rmse_train_mlp_weight_reg.describe()

In [ ]:
rmse_test_mlp_weight_reg.describe()

### Plotting the Simple MLPs

In [ ]:
sns.set_style("white")

#### If not all models were run: Read saved files

In [ ]:
hist_simple_mlp = pd.read_pickle("../final_data/hist_simple_mlp_train.pkl")
hist_simple_dropout = pd.read_pickle("../final_data/hist_simple_mlp_drop.pkl")
hist_simple_weight_reg = pd.read_pickle("../final_data/hist_simple_mlp_weight.pkl")

In [ ]:
fig_simple_mlps, ax = plt.subplots(figsize=(10, 5))
ax.plot(hist_simple_mlp['loss'], label='SimpleMLP_Loss', color='dimgrey' )
ax.plot(hist_simple_mlp['val_loss'], label='SimpleMLP_Validation_Loss', color='black' )
ax.plot(hist_simple_dropout['loss'], label='SimpleDropout_Loss', color='indianred')
ax.plot(hist_simple_dropout['val_loss'], label='SimpleDropout_Validation_Loss', color='red')
ax.plot(hist_simple_weight_reg['loss'], label='SimpleWeightReg_Loss', color='c')
ax.plot(hist_simple_weight_reg['val_loss'], label='SimpleWeightReg_Validation_Loss', color='blue')
ax.set_ylim(0.001, 0.004)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper center',prop={'size': 12} )
ax.grid()

### Plotting tuned models

#### If not all models were run: Read saved files

In [ ]:
hist_dropout_train = pd.read_pickle("../final_data/hist_mlp_dropout_hyper_train.pkl")
hist_weight_reg_train = pd.read_pickle("../final_data/hist_hypermodel_mlp_weight_reg.pkl")

In [ ]:
fig_final_mlps, ax = plt.subplots(figsize=(10, 5))
ax.plot(hist_dropout_train['loss'][:300], label='MLPDropout_Loss', color='indianred' )
ax.plot(hist_dropout_train['val_loss'][:300], label='MLPDropout_Validation_Loss', color='red' )
ax.plot(hist_weight_reg_train['loss'][:300], label='MLPWeightDecay_Loss', color='c')
ax.plot(hist_weight_reg_train['val_loss'][:300], label='MLPWeightDecay_Validation_Loss', color='blue')
ax.set_ylim(0.001, 0.004)
ax.set(xlabel=' Epochs', ylabel='Score')
ax.set_xlabel('Epochs', fontsize=15)
ax.set_ylabel('Score', fontsize=15)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
ax.legend(loc='upper left',prop={'size': 13} )
ax.grid()

### Graphical Evaluation

#### Load best model if needed

In [ ]:
mlp_weight_reg_train = keras.models.load_model("../final_data/models/hypermodel_mlp_weight_reg_train")
mlp_weight_reg_test = keras.models.load_model("../final_data/models/mlp_weight_reg_test")

In [ ]:
y_pred_train = mlp_weight_reg_test.predict(all_train_x)
y_train_unsc = scaler_y.inverse_transform(all_train_y)
y_pred_train_unsc = scaler_y.inverse_transform(y_pred_train)
np.sqrt(mean_squared_error(y_train_unsc, y_pred_train_unsc))                       

In [ ]:
y_pred_valid = mlp_weight_reg_train.predict(valid_x)
y_valid_unsc = scaler_y_train.inverse_transform(valid_y)
y_pred_valid_unsc = scaler_y_train.inverse_transform(y_pred_valid)
np.sqrt(mean_squared_error(y_valid_unsc, y_pred_valid_unsc))                       

In [ ]:
y_pred_test_final = mlp_weight_reg_test.predict(test_x)
y_test_unsc = scaler_y.inverse_transform(test_y)
y_pred_test_unsc_final = scaler_y.inverse_transform(y_pred_test_final)
np.sqrt(mean_squared_error(y_test_unsc, y_pred_test_unsc_final))   

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_train_unsc, y_train_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_valid_unsc, y_valid_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.set(xticks=[]) 
ax1.xaxis.grid(True)
ax1.yaxis.grid(True)

In [ ]:
fig, ax1 = plt.subplots(1,  figsize=(5, 5),  sharex=True, sharey=True)
plt.xlim(550, 900)
plt.ylim(550, 900)

ax1.scatter( y_pred_test_unsc_final, y_test_unsc,  c= 'dodgerblue',s=1)
ax1.plot([550,900],[550,900], c='black')
ax1.grid()
ax1.set_xlabel('Predicted Temperatures [°C] ', fontsize=15)
ax1.set_ylabel('Measured Temperature [°C]',fontsize=15)
ax1.set(yticks=[]) 
ax1.xaxis.grid(True)
ax1.set(xticks=[]) 
ax1.yaxis.grid(True)